### All necessary imports

In [1]:
from lxml.html import parse
from urllib.request import urlopen
from pandas.io.parsers import TextParser
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup #for web scraping
import csv  
import os
import sys
import logging
from pandas.io.parsers import TextParser

### Storing the cik and acc number in variables

In [2]:
ErrorLog = 'errorLogging.log'
logging.basicConfig(filename=ErrorLog,level=logging.DEBUG, filemode = 'w')

In [3]:
file_obj = open("cikInformation.txt","r" )
cik = file_obj.readline().rstrip()
acc = file_obj.readline().rstrip()
if((cik is None or cik is "") or (acc is None or acc is "")):
    logging.info("Cik or access number was left blank. Assigning a IBM's CIK and access key automatically")
    cik = "51143"
    acc = "0000051143-13-000007"
    accNoDash = acc.replace('-', '')
else:
    accNoDash = acc.replace('-', '')

In [4]:
print(cik)
print(accNoDash)

51143
000005114313000007


### Parsing the url 

In [5]:
#Checking if log works: logging.debug('This message should go to the log file')

try:
    url = 'https://www.sec.gov/Archives/edgar/data/'+cik+'/'+accNoDash+'/'+acc+'-index.htm'
    parsed = parse(urlopen(url))
    doc = parsed.getroot()
except:
    logging.critical("Invalid company's url")
    logging.info("Validate that cik number and acces number are correctly inputted")
    exit()

In [6]:
url = 'https://www.sec.gov/Archives/edgar/data/'+cik+'/'+accNoDash+'/'+acc+'-index.htm'
parsed = parse(urlopen(url))
doc = parsed.getroot()

In [7]:
#links = doc.findall('.//a')
#url = [lnk.get('href')for lnk in doc.findall('.//a')]
#url[10:23]

In [8]:
tables = doc.findall('.//table')

In [9]:
Doc=[]
for t in range(len(tables)):
    example = tables[t]
    
    rows = example.findall('.//tr')
    
    def _unpack(row, kind='td'):
        elts = row.findall('.//%s' % kind)
        return [val.text_content() for val in elts]

    def parse_options_data(table):
        rows=table.findall('.//tr')
        header = _unpack(rows[0], kind ='th')
        data = [_unpack(r) for r in rows[1:]]
        return TextParser(data, names = header).get_chunk()

    example_data = parse_options_data(example)
    
    
    for r in range(len(example_data)):
        d= example_data['Description'][r]
        if d =='10-Q' :
            Doc.append(example_data['Document'][r])

In [10]:
Doc

['ibm13q3_10q.htm']

https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm

## 10-Q Links

In [11]:
for link in Doc:
    try:
        L=[] #For storing all the 10-Q file links
        l = 'https://www.sec.gov/Archives/edgar/data/'+cik+'/'+accNoDash+'/'+link+''
        L.append(l)
    except:
        logging.critical("Invalid 10Q url")
        logging.info("Validate that cik number and acces number are correctly inputted")
        exit()

In [12]:
L

['https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm']

### Traversing each 10-Q links

In [13]:
#for l in L:
    #p = parse(urlopen(l))
    #q_link= p.getroot()
    #tables_2 = q_link.findall('.//table')

### Traversing through all tables in 10-Q document and storing the tables in the list -> f_tables[ ]

In [14]:
for l in L:
    page = urllib.request.urlopen(l)
    soup = BeautifulSoup(page,"lxml")
    # Grabing all the tables considering the table tags
    table_2 = soup.select('div table')
        
    f_tables=[]
        
    for t in table_2:
        for row in t.find_all('tr'):
            for cell in row.findAll('td') :
                flag = 0
                if ('$' in cell.get_text().strip() or '%' in cell.get_text().strip()):
                    f_tables.append(t)
                    flag=1
                    break
            if(flag==1):
                break 

In [53]:
# n_columns = 0
# n_rows=0
# column_names = []
    

# for row in f_tables[4].find_all('tr'):
#             td_tags = row.find_all('td')
#             print(len(td_tags))
#             if len(td_tags) > 0:
#                 n_rows+=1
#                 if n_columns == 0:
#                         # Set the number of columns for our table
#                     n_columns = len(td_tags)
        
#         # Handle column names if we find them
#             th_tags = row.find_all('th') 
#             if len(th_tags) > 0 and len(column_names) == 0:
#                 for th in th_tags:
#                     column_names.append(th.get_text())
        
# if len(column_names) > 0 and len(column_names) != n_columns:
#                     raise Exception("Column titles do not match the number of columns")
    
# columns = column_names if len(column_names) > 0 else range(0,n_columns)
# df = pd.DataFrame(columns = columns,index= range(0,n_rows))

# row_marker = 0
# for row in f_tables[4].find_all('tr'):
#     column_marker = 0
#     columns = row.find_all('td')
#     for column in columns:
#         df.iat[row_marker,column_marker] = column.get_text()
#         column_marker += 1
#     if len(columns) > 0:
#         row_marker += 1
        
# for col in df:
#     try:
#         df[col] = df[col].astype(float)
#     except ValueError:
#         pass

# #Removing unnecessary blanks
# #df.replace('\n\xa0\n', '')
# for i in range(n_columns):
#     for j in range(n_rows):
#         if(type(df[i][j]) != float):
#             x = df[i][j]
#             df[i][j] = x.strip('\n')
#         else:
#             x = str(df[i][j])
#             df[i][j] = x.strip('\n')

# # Converting the dataframe in csv file

# with open('1.csv', 'a') as csvfile:
#     df.to_csv(csvfile, header=False)

In [54]:
len(f_tables)

101

### Storing the table in a CSV files

In [66]:
for tt in f_tables:
    records = []
    for tr in tt.find_all('tr'):
        rowString=[]
        for td in tr.findAll('td'):
            p = td.find_all('p')
            if len(p)>0:
                for ps in p:
                    ps_text = ps.get_text().replace("\n"," ") 
                    ps_text = ps_text.replace("\xa0","")                 
                    rowString.append(ps_text)
            else:
                td_text=td.get_text().replace("\n"," ")
                td_text = td_text.replace("\xa0","")
                rowString.append(td_text)
        records.append(rowString)        
    with open((str(f_tables.index(tt)+1)+'_'+str(cik)+'.csv'), 'w')as f:
        writer = csv.writer(f)
        writer.writerows(records)